In [1]:
import pandas as pd
import numpy as np
import sqlite3 

In [2]:
data = pd.read_csv('final_data.csv')


In [3]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'imdb_id', 'original_title', 'cast',
       'homepage', 'tagline', 'keywords', 'overview', 'runtime', 'genres',
       'release_year', 'tags', 'averageRating'],
      dtype='object')

In [4]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
tfv = TfidfVectorizer(min_df=3,max_features=None, strip_accents='unicode',analyzer='word',token_pattern=r'\w{1,}',ngram_range=(1,3),stop_words='english')


In [6]:
data['tags']= data['tags'].fillna(' ')

In [7]:
tfv_matrix= tfv.fit_transform(data['tags'])

In [8]:
tfv_matrix

<10850x31006 sparse matrix of type '<class 'numpy.float64'>'
	with 445596 stored elements in Compressed Sparse Row format>

In [9]:
tfv_matrix.shape

(10850, 31006)

In [10]:
from sklearn.metrics.pairwise import sigmoid_kernel
#compute sigmoid kernel
sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [11]:
sig[0]

array([0.7616077 , 0.76159532, 0.76159526, ..., 0.76159416, 0.76159426,
       0.76159416])

In [12]:
indices = pd.Series(data.index, index= data['original_title']).drop_duplicates()

In [13]:
indices

original_title
Jurassic World                      0
Mad Max: Fury Road                  1
Insurgent                           2
Star Wars: The Force Awakens        3
Furious 7                           4
                                ...  
The Endless Summer              10845
Grand Prix                      10846
Beregis Avtomobilya             10847
What's Up, Tiger Lily?          10848
Manos: The Hands of Fate        10849
Length: 10850, dtype: int64

In [14]:
def give_rec(title, sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    sig_scores = sorted(sig_scores, key= lambda x: x[1], reverse= True)
    sig_scores = sig_scores[1:11]
    print(sig_scores)
    movie_indices = [i[0] for i in sig_scores]
    return data['original_title'].iloc[movie_indices]

In [15]:
give_rec('Avengers: Age of Ultron')

[(1914, 0.7615979820369818), (5414, 0.7615973678780326), (4353, 0.7615972381043888), (2870, 0.7615970143338007), (5904, 0.7615959677770722), (629, 0.7615959607967727), (9237, 0.7615958438203598), (2226, 0.7615957056042705), (2880, 0.7615956989111381), (8214, 0.7615956983763374)]


1914                                           Iron Man 2
5414                                           Iron Man 3
4353                                         The Avengers
2870                                             Iron Man
5904                     Phineas and Ferb: Mission Marvel
629                   Captain America: The Winter Soldier
9237    The Toxic Avenger Part III: The Last Temptatio...
2226                         Scrat's Continental Crack-Up
2880                                  The Incredible Hulk
8214                     Darkman II: The Return of Durant
Name: original_title, dtype: object